# Deliverable 1: Preprocessing the Data for a Neural Network

### 1. Import & Inspect the Data

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf


# Connect to Google Drive
from google.colab import drive 
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("gdrive/My Drive/Colab Notebooks/Neural_Network_Charity_Analysis/Resources/charity_data.csv")
application_df.head()

*   **EIN and NAME** — Identification columns
*   **APPLICATION_TYPE** — Alphabet Soup application type
*   **AFFILIATION** — Affiliated sector of industry
*   **CLASSIFICATION** — Government organization classification
*   **USE_CASE** — Use case for funding
*   **ORGANIZATION** — Organization type
*   **STATUS** — Active status
*   **INCOME_AMT** — Income classification
*   **SPECIAL_CONSIDERATIONS** — Special consideration for application
*   **ASK_AMT** — Funding amount requested
*   **IS_SUCCESSFUL** — Was the money used effectively? (1=yes 0=no ?)

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["EIN","NAME"], axis=1, inplace=True)
application_df.head()

In [ ]:
application_df.dtypes

## 2. Preprocessing

### A) Encode categorical data


In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
app_type_vc = application_df["APPLICATION_TYPE"].value_counts()
app_type_vc

In [ ]:
# Visualize the value counts of APPLICATION_TYPE
app_type_vc.plot.density()

In [ ]:
# Determine which values to replace if counts are less than 500?
replace_application = list(app_type_vc[app_type_vc < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
class_vc = application_df["CLASSIFICATION"].value_counts()
class_vc

In [ ]:
# Visualize the value counts of CLASSIFICATION
class_vc.plot.density()

In [ ]:
# Determine which values to replace if counts are less than 1800?
replace_class = list(class_vc[class_vc < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

In [ ]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat, 1)
application_df.head()

In [ ]:
application_df.columns

### B) Split into training and testing data

In [ ]:
# Split our preprocessed data into our features (X) and target (y) arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
application_df.columns

### C) Standardize all numerical data

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Deliverable 2: Compile, Train and Evaluate the Model

### 1. Define Deep Learning Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add( tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu") )

# Second hidden layer
nn.add( tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu") )

# Output layer
nn.add( tf.keras.layers.Dense(units=1, activation="sigmoid") )

# Check the structure of the model
nn.summary()

### 2. Compile & Train the Model

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period = 5)

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

### 3. Evaluate the Model

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

### 4. Save the Model

In [ ]:
nn.save("AlphabetSoupCharity.h5")